## Index and Selection: Part II

### Reindexing ###

索引一个可能不存在的元素时, `reindex()` 是一种很自然合理的选择. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
s = pd.Series([1,2,3])
s

0    1
1    2
2    3
dtype: int64

In [4]:
s.reindex([1, 2, 3])

1    2.0
2    3.0
3    NaN
dtype: float64

如果希望在一组label中, 找到有效的key, 可以使用下面的方法

In [5]:
labels = [1,2,3]
s.loc[s.index.intersection(labels)]

1    2
2    3
dtype: int64

### 随机选择样本 ###

Series, DataFrame 和 Panel对象都拥有 `sample()` 方法.

随机选取一些样本的 sample() 方法提供了以下可供选择的参数:

1. 希望sample出的samples的数量; 这样的数目, 既可用数量n表示, 也可以样本占总数的比值来表示(frac)
2. 指定是不是可放回随机抽样. 即每一行(假设是在选择行)是否被允许出现超过一次. 使用 replace=True 指定可放回抽样
3. 指定每行被选中的权重. 这里Pandas会normalize加起来不等于1的权重列表.

In [7]:
df = pd.DataFrame({'col1':[9,8,7,6], 'weight_column':[0.5, 0.4, 0.1, 0]})

In [11]:
df.sample(n=1, weights='weight_column')

,col1,weight_column
0,9,0.5


以上代码展示了使用 `DataFrame` 对象中的一列作为每行被选中的权重.

### 快速地获取和设置一个标量数值 ###

所有牵扯到 `[]` 的索引方法都涉及到了处理多种情况, 比如 list of labels, callable, 或者 boolean array. 因此如果要快速地获取或者设置一个标量数值(scalar value), 我们需要更specialized的方法: `at`, `iat`

1. `at` 类似于 `.loc` 方法, 使用 label 进行索引
2. `iat` 类似于 `.iloc` 方法, 使用 integer 进行索引

In [12]:
dates = pd.date_range('1/1/2000', periods=8)
df = pd.DataFrame(np.random.randn(8,4), index=dates, columns=['A','B','C','D'])

In [13]:
dates

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07', '2000-01-08'],
              dtype='datetime64[ns]', freq='D')

In [14]:
df

,A,B,C,D
2000-01-01,0.866928,-1.673241,-0.578069,0.956704
2000-01-02,-1.040658,1.119744,0.558309,0.554854
2000-01-03,-0.006972,-0.928737,0.140272,-0.794356
2000-01-04,-0.226575,-1.457587,0.341986,0.127456
2000-01-05,-1.322853,-0.535144,-1.941909,-2.214788
2000-01-06,0.283675,-0.585195,-1.312493,-0.159598
2000-01-07,0.004550,0.862819,-2.599116,-0.510084
2000-01-08,-2.579770,-0.092715,0.824241,-0.763059


In [18]:
df.at[dates[5], 'B']

-0.5851946111001831

In [48]:
# .at() 与 .loc() 获取一个scalar value 有什么区别?
df.loc[dates[5], 'B']

-0.5851946111001831

以上是使用 `at` 方法, 接收 label 参数, 获取scalar value. 其取到的值和以下语句相同

In [19]:
df.iat[5, 1]

-0.5851946111001831

`at` 或者 `iat` 也可以enlarge Pandas 对象. 如以下所示

In [23]:
df.at[dates[-1]+1, 'F'] = 7
df

,A,B,C,D,F
2000-01-01,0.866928,-1.673241,-0.578069,0.956704,NaN
2000-01-02,-1.040658,1.119744,0.558309,0.554854,NaN
2000-01-03,-0.006972,-0.928737,0.140272,-0.794356,NaN
2000-01-04,-0.226575,-1.457587,0.341986,0.127456,NaN
2000-01-05,-1.322853,-0.535144,-1.941909,-2.214788,NaN
2000-01-06,0.283675,-0.585195,-1.312493,-0.159598,NaN
2000-01-07,0.004550,0.862819,-2.599116,-0.510084,NaN
2000-01-08,-2.579770,-0.092715,0.824241,-0.763059,NaN
2000-01-09,NaN,NaN,NaN,NaN,7.0


### Boolean 索引 ###
使用一组boolean 向量过滤数据也是一种常见的操作. 操作符是 `&` (and), `|` (or), `~` (not).

由于Python规定的符号优先级原因, 在Pandas中使用这三个操作符时, 务必使用括号.

最基本的使用Boolean list索引的方式已经有所介绍. 以下是更为复杂的, 结合了三种操作符的索引方式

In [27]:
df2 = pd.DataFrame({'a':['one', 'one','two', 'three', 'two', 'one', 'six'],
                   'b':['x', 'y', 'y', 'x', 'y', 'x', 'x'],
                   'c':np.random.randn(7)})
df2

,a,b,c
0,one,x,-0.268765
1,one,y,1.427269
2,two,y,-1.327119
3,three,x,0.791206
4,two,y,1.218323
5,one,x,0.079549
6,six,x,-0.878306


In [30]:
condition = df2['a'].map(lambda x : x.startswith('t'))
df2[condition]

,a,b,c
2,two,y,-1.327119
3,three,x,0.791206
4,two,y,1.218323


In [31]:
condition

0    False
1    False
2     True
3     True
4     True
5    False
6    False
Name: a, dtype: bool

在上面的代码中, `df2['a'].map(lambda x: x.startswith('t'))` 返回一个boolean series. 该series作为参数输入进 `[]` 进行索引. 上面的代码和下面的代码完成一样的功能, 但是下面的代码运行速度更慢

In [32]:
df2[[x.startswith('t') for x in df2['a']]]

,a,b,c
2,two,y,-1.327119
3,three,x,0.791206
4,two,y,1.218323


此时我们引入第二个条件:
在找到 `a` 列中以 ‘t’ 开头的所有行之后, 再检索 `b` 列中所有等于 ‘x’ 的行. 并符合这些条件的行的全部列. 下面的代码所示

In [33]:
df2[condition & (df2['b'] == 'x')]

,a,b,c
3,three,x,0.791206


在上面的条件中, 我们再选出结果中的 'b' 列与 'c' 列. 此时可以使用 loc 索引

In [34]:
df2.loc[condition & (df2['b'] == 'x'), 'b':'c']

,b,c
3,x,0.791206


### 使用 isin() 索引 ###

1. 对于Series, isin() 返回一个boolean vector, 该vector的长度与Series相等, 如果Series中的元素位于传入 `isin()` 函数的list中, 则元素对应位置的 isin() 返回值为true, 否则为false.

In [36]:
s1 = pd.Series(np.arange(5), index=np.arange(5)[::-1], dtype='int64')
s1

4    0
3    1
2    2
1    3
0    4
dtype: int64

In [37]:
s1.isin([2,4,6])

4    False
3    False
2     True
1    False
0     True
dtype: bool

In [39]:
# 利用isin() 返回的boolean vector进行索引
s1.loc[s1.isin([2,4,6])]

2    2
0    4
dtype: int64

Series的Index也有isin() 方法, 可以使用这个方法检索Index

In [41]:
s1[s1.index.isin([2,4,6])]

4    0
2    2
dtype: int64

In [42]:
# 以上的代码和reindex进行比较
s1.reindex([2,4,6])

2    2.0
4    0.0
6    NaN
dtype: float64

使用 isin() 索引与reindex索引, 区别在于 reindex() 会给不存在的Index自动补值为0. 而这很多时候不是想要的. 因此使用`Series.loc[Series.index.isin([some,list])` 有时候是一种更好的方法

Series的MultiIndex也可以使用Series. 可以给 isin() 传入一组 MultiIndex tuple. 另一种选择是在调用MultiIndex的isin() 方法时, 传入Level参数

In [46]:
# 构建一个Multi Index
s_mi = pd.Series(np.arange(6), index=pd.MultiIndex.from_product([[0,1], ['a', 'b', 'c']]))
s_mi

0  a    0
   b    1
   c    2
1  a    3
   b    4
   c    5
dtype: int64

In [50]:
s_mi.iloc[s_mi.index.isin([(1,'a'), (2, 'b'), (0, 'c')])]

0  c    2
1  a    3
dtype: int64

In [53]:
# 调用MultiIndex的isin() 方法时, 传入level指定特定level的index被用于判断是否存在于参数列表中.
s_mi.index.isin(['a','c','e'], level=1)

array([ True, False,  True,  True, False,  True])

`DataFrame` 也有isin() 方法. isin() 方法可以传入两种不同的数据类型

1. array. 如果传入array, 则 isin() 方法返回值是和原DataFrame shape相同的DataFrame. 返回的DataFrame每一个元素都是Boolean值. 当原DataFrame某一位置的元素位于isin() 方法传入的参数时, 则在返回值中对应位置的值为True

2. dict. dict 的key为列明,  value为一个list. isin() 方法的返回值则是原DataFrame shape相同的DataFrame. 返回的DataFrame每一个元素都是Boolean值.

In [55]:
df = pd.DataFrame({'vals': [1, 2, 3, 4], 'ids': ['a', 'b', 'f', 'n'],'ids2': ['a', 'n', 'c', 'n']})
df

,ids,ids2,vals
0,a,a,1
1,b,n,2
2,f,c,3
3,n,n,4


In [56]:
# isin() 传入 array
df.isin(['a','b',1,3])

,ids,ids2,vals
0,True,True,True
1,True,False,False
2,False,False,True
3,False,False,False


In [57]:
# isin() 传入 dict
values = {'ids': ['a','b'], 'vals': [1, 3]}
df.isin(values)

,ids,ids2,vals
0,True,False,True
1,True,False,False
2,False,False,True
3,False,False,False


将DataFrame的 isin() 方法与 all(), any() 方法结合, 可以快速选择符合某一条件的数据子集

In [58]:
values = {'ids': ['a', 'b'], 'ids2': ['a', 'c'], 'vals': [1, 3]}

In [59]:
df.isin(values)

,ids,ids2,vals
0,True,True,True
1,True,False,False
2,False,True,True
3,False,False,False


In [63]:
row_mask = df.isin(values).all(1)
row_mask

0     True
1    False
2    False
3    False
dtype: bool

In [64]:
# 使用row_mask索引
df.loc[row_mask]

,ids,ids2,vals
0,a,a,1
